# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [162]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

In [163]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
spaceship.shape

(8693, 14)

In [164]:
spaceship = spaceship.dropna()

In [165]:
spaceship

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [166]:
def cabin_category(cabin):
    return cabin.split('/')[0]

spaceship["Cabin"] = spaceship["Cabin"].apply(cabin_category)

In [167]:
spaceship = spaceship.drop(['PassengerId', 'Name'], axis=1)

In [168]:
spaceship

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,G,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,G,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,E,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [169]:
spaceship_dummies = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Transported'], drop_first=True).astype(int)

spaceship_dummies

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,Transported_True
0,39,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
1,24,109,9,25,549,44,0,0,0,0,0,0,0,1,0,0,0,1,0,1
2,58,43,3576,0,6715,49,1,0,0,0,0,0,0,0,0,0,0,1,1,0
3,33,0,1283,371,3329,193,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,16,303,70,151,565,2,0,0,0,0,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41,0,6819,0,1643,74,1,0,0,0,0,0,0,0,0,0,0,0,1,0
8689,18,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0
8690,26,0,0,1872,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
8691,32,0,1049,0,353,3235,1,0,0,0,0,0,1,0,0,0,0,0,0,0


In [170]:
features = spaceship_dummies.drop("Transported_True", axis = 1)
target = spaceship_dummies['Transported_True']

**Perform Train Test Split**

In [171]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20)

In [172]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [173]:
pd.DataFrame(X_train_norm, columns=X_train.columns)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,0.354430,0.000000,0.000000,0.000000,0.029855,0.000925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.632911,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.075949,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.506329,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.683544,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,0.354430,0.000000,0.000034,0.000979,0.039049,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5280,0.265823,0.055444,0.000000,0.004978,0.000179,0.000289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5281,0.658228,0.081552,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5282,0.341772,0.002016,0.000000,0.081613,0.001607,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [217]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=5),
                               n_estimators=200,
                               max_samples = 1500)

In [218]:
bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=5), max_samples=1500,
                 n_estimators=200)

In [219]:
pred = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

MAE 0.2863233852449882
RMSE 0.3632067387759223
R2 score 0.47048010635151993


In [220]:
pred = bagging_reg.predict(X_train_norm)

print("MAE", mean_absolute_error(pred, y_train))
print("RMSE", root_mean_squared_error(pred, y_train))
print("R2 score", bagging_reg.score(X_train_norm, y_train))

MAE 0.29100205872918655
RMSE 0.367885306752526
R2 score 0.4586241534852744


- Random Forests

In [221]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=5)

In [222]:
forest.fit(X_train_norm, y_train)

RandomForestRegressor(max_depth=5)

In [223]:
pred = forest.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", forest.score(X_test_norm, y_test))

MAE 0.28510144865487835
RMSE 0.3654124847090804
R2 score 0.4640290535289132


In [224]:
# CHECK FOR OVERFIT

pred = forest.predict(X_train_norm)

print("MAE", mean_absolute_error(pred, y_train))
print("RMSE", root_mean_squared_error(pred, y_train))
print("R2 score", forest.score(X_train_norm, y_train))

MAE 0.2875524428941716
RMSE 0.36707847804236704
R2 score 0.46099618947595455


- Gradient Boosting

In [225]:
gb_reg = GradientBoostingRegressor(max_depth=5,
                                   n_estimators=100)

In [226]:
gb_reg.fit(X_train_norm, y_train)

GradientBoostingRegressor(max_depth=5)

In [227]:
pred = gb_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", gb_reg.score(X_test_norm, y_test))

MAE 0.26643834857614107
RMSE 0.36000448680325886
R2 score 0.4797760856098867


In [228]:
# CHECK OVERFIT - tipical case of overfit

pred = gb_reg.predict(X_train_norm)

print("MAE", mean_absolute_error(pred, y_train))
print("RMSE", root_mean_squared_error(pred, y_train))
print("R2 score", gb_reg.score(X_train_norm, y_train))

MAE 0.23414937352901136
RMSE 0.31986103390507775
R2 score 0.5907424838682136


- Adaptive Boosting

In [243]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                            n_estimators=200)

In [244]:
ada_reg.fit(X_train_norm, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=4),
                  n_estimators=200)

In [245]:
pred = ada_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", ada_reg.score(X_test_norm, y_test))

MAE 0.3466681725120905
RMSE 0.3916166520541939
R2 score 0.38440258684400463


In [246]:
pred = ada_reg.predict(X_train_norm)

print("MAE", mean_absolute_error(pred, y_train))
print("RMSE", root_mean_squared_error(pred, y_train))
print("R2 score", ada_reg.score(X_train_norm, y_train))

MAE 0.35124790753044693
RMSE 0.39368052622608685
R2 score 0.38004258925044976


Which model is the best and why?

In [ ]:
# They all performed really bad. There is a slight preference for Bagging and Pasting